## Importing

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

In [3]:
from utils.utils import *
import optuna
from optuna.storages import JournalStorage
from optuna.storages.journal import JournalFileBackend
from experiments.experiments_gkan import ExperimentsGKAN
from experiments.experiments_mpnn import ExperimentsMPNN
import sympytorch

import warnings
warnings.filterwarnings("ignore")


Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


In [4]:
import random

def set_pytorch_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    torch.use_deterministic_algorithms(True)
set_pytorch_seed(0)

## Utils

In [5]:
from models.utils.MPNN import MPNN
from models.baseline.MPNN_ODE import MPNN_ODE
from train_and_eval import eval_model
from datasets.SyntheticData import SyntheticData
from sympy import symbols, sin, summation, simplify
import networkx as nx
from torch_geometric.utils import from_networkx
from utils.utils import integrate
from torch_geometric.data import Data
from experiments.experiments_mpnn import activations
from models.utils.MLP import MLP

import json
import torch
from models.baseline.LLC import LLC_ODE
from models.utils.MPNN import MPNN
from models.baseline.LLC_Conv import Q_inter, Q_self
from experiments.experiments_mpnn import activations

In [12]:
from sympy import latex
from torch.utils.data import DataLoader
from post_processing import get_model, make_callable, get_symb_test_error, get_test_set, integrate_test_set, plot_predictions


def build_model_from_file_mpnn(model_path, message_passing=False, include_time=False, method='dopri5', adjoint=True, atol=1e-5, rtol=1e-5):
    best_params_file = f"{model_path}/best_params.json"
    best_state_path = f"{model_path}/mpnn/state_dict.pth"
    with open(best_params_file, 'r') as f:
        best_hyperparams = json.load(f)
    
    in_dim = 1
    
    hidden_layers = [best_hyperparams["hidden_dims_g_net"] for _ in range(best_hyperparams["n_hidden_layers_g_net"])]
    hidden_layers = [2*in_dim] + hidden_layers + [in_dim]    
    # g_net
    g_net = MLP(
        hidden_layers=hidden_layers,
        af = activations[best_hyperparams['af_g_net']],
        dropout_rate=best_hyperparams['drop_p_g_net'],
    )
    
    time_dim = 1 if include_time else 0
    in_dim_h = 2 if message_passing else 1
    in_dim_h += time_dim
    hidden_layers = [best_hyperparams["hidden_dims_h_net"] for _ in range(best_hyperparams["n_hidden_layers_h_net"])]
    hidden_layers = [in_dim_h] + hidden_layers + [in_dim] 
    
    
    # h_net
    h_net = MLP(
        hidden_layers=hidden_layers,
        af = activations[best_hyperparams['af_h_net']],
        dropout_rate=best_hyperparams['drop_p_h_net'],
    )
    
    mpnn = MPNN(
        h_net=h_net,
        g_net=g_net,
        message_passing=message_passing,
        include_time=include_time
    )
    
    model = MPNN_ODE(
        conv=mpnn,
        model_path='./saved_models_optuna/tmp',
        integration_method=method,
        adjoint=adjoint,
        atol=atol,
        rtol=rtol
    )
    
    model = model.to(torch.device('cuda'))
    model.load_state_dict(torch.load(best_state_path, weights_only=False, map_location=torch.device('cuda')))
    
    return model


def build_model_from_file_llc(model_path, message_passing=False, include_time=False, method='dopri5', adjoint=True, atol=1e-5, rtol=1e-5):

    best_params_file = f"{model_path}/best_params.json"
    best_state_path = f"{model_path}/llc/state_dict.pth"

    with open(best_params_file, 'r') as f:
        best_hyperparams = json.load(f)

    in_dim = 1
    time_dim = 1 if include_time else 0

    # === g_net config (for Q_inter) ===
    def build_q_layer_config(prefix):
        n_layers = best_hyperparams[f'n_hidden_layers_{prefix}']
        hidden_dim = best_hyperparams[f'hidden_dims_{prefix}']
        activation = activations[best_hyperparams[f'af_{prefix}']]
        dropout = best_hyperparams[f'drop_p_{prefix}']

        if prefix == "g0":
            input_dim = 2 * in_dim
        else:
            input_dim = in_dim 

        hidden_layes = [hidden_dim] * n_layers
        layers = [input_dim] + hidden_layes + [in_dim]
        return {
            f'hidden_layers_{prefix}': layers,
            f'af_{prefix}': activation,
            f'dr_{prefix}': dropout
        }

    g0_config = build_q_layer_config("g0")
    g1_config = build_q_layer_config("g1")
    g2_config = build_q_layer_config("g2")
    g_net = Q_inter(**{**g0_config, **g1_config, **g2_config})

    # === h_net config (for Q_self) ===
    in_dim_h = 2 * in_dim if message_passing else in_dim
    in_dim_h += time_dim
    hidden_dim_h = best_hyperparams["hidden_dims_h_net"]
    n_layers_h = best_hyperparams["n_hidden_layers_h_net"]
    hidden_layers_h = [hidden_dim_h] * n_layers_h
    layers_h = [in_dim_h] + hidden_layers_h + [in_dim]

    h_net = Q_self(
        hidden_layers=layers_h,
        af=activations[best_hyperparams['af_h_net']],
        dropout_rate=best_hyperparams['drop_p_h_net']
    )

    # === Full MPNN and ODE wrapper ===
    mpnn = MPNN(
        h_net=h_net,
        g_net=g_net,
        message_passing=message_passing,
        include_time=include_time
    )

    model = LLC_ODE(
        conv=mpnn,
        model_path='./saved_models_optuna/tmp',
        adjoint=adjoint,
        integration_method=method,
        atol=atol,
        rtol=rtol
    )

    model = model.to(torch.device('cuda'))
    model.load_state_dict(torch.load(best_state_path, weights_only=False, map_location=torch.device('cuda')))

    return model


def valid_symb_model(
    config,
    model_path_gkan,
    device='cuda',
    atol=1e-5,
    rtol=1e-5,
    method='dopri5',
    sample_size=10000
):
    seed = 9999
    graph = nx.barabasi_albert_graph(100, 3, seed=seed)

    # Prepare validation/test set
    valid_set = integrate_test_set(
        graph=graph,
        dynamics=config['name'],
        seed=seed,
        device=device,
        input_range=config['input_range'],
        t_span=(0, 1),
        **config['integration_kwargs']
    )

    # Helper to compute validation loss
    def evaluate_model(g_symb, h_symb, is_symb=True):
        errs = get_symb_test_error(
            g_symb=g_symb,
            h_symb=h_symb,
            test_set=[valid_set],
            message_passing=False,
            include_time=False,
            method=method,
            atol=atol,
            rtol=rtol,
            is_symb=is_symb
        )
        return errs[0]

    # Helper to fit model for current config
    def fit_single_model(param1, param2):
        print(f"Fitting black-box model with {param1} and {param2} iterations")
        pysr_model = lambda: get_pysr_model(
            model_selection=param1, 
            n_iterations=param2,
            # parallelism="serial",
            # random_state = seed,
            # deterministic = True
        )
        _, g_symb, h_symb, _ = fit_mpnn(
            device=device,
            model_path=model_path_gkan,
            pysr_model=pysr_model,
            sample_size=sample_size,
            message_passing=False,
            verbose=False
        )
        
        return g_symb, h_symb

    param_grid = (["score", "accuracy"], [50, 100, 200])
    search_space = [(mod, val) for mod in param_grid[0] for val in param_grid[1]]
    valid_losses = []
    
    for mod, val in search_space:
        g_symb, h_symb = fit_single_model(mod, val)
        try:
            loss = evaluate_model(g_symb, h_symb, is_symb=True)
        except AssertionError:
            loss = 1e8
        valid_losses.append({'model_selection': mod, 'param': val, 'valid_loss': loss})
    
    best = min(valid_losses, key=lambda x: x['valid_loss'])    
    
    print(f"Refitting best model with {best}")
    
    gkan_symb, symb_g, symb_h, exec_time = fit_mpnn(
        model_path=model_path_gkan,
        device=device,
        pysr_model=lambda: get_pysr_model(
            model_selection=best['model_selection'],
            n_iterations=best['param'],
            # parallelism="serial",
            # random_state = seed,
            # deterministic = True
        ),
        sample_size=sample_size,
        message_passing=False,
        verbose=True,
        include_time=False
    )
    
    return gkan_symb, symb_g, symb_h, exec_time


def post_process_mpnn(
    config,
    model_path, 
    test_set, 
    device='cuda',
    sample_size=10000,
    message_passing=False, 
    include_time=False,
    atol=1e-5,
    rtol=1e-5,
    method='dopri5',
    scaler=None,
    inverse_scale=False,
    adjoint=True,
    eval_model=True,
    model_type="MPNN",
    res_file_name="post_process_res.json"
):
    
    results_dict = {}
    
    def print_symb_error(g_symb, h_symb, txt="symbolic formula", is_symb=True):
        try:
            test_losses_symb = get_symb_test_error(
                g_symb=g_symb,
                h_symb=h_symb,
                test_set=test_set,
                message_passing=message_passing,
                include_time=include_time,
                atol=atol,
                rtol=rtol,
                method=method,
                scaler=scaler,
                inverse_scale=inverse_scale,
                is_symb=is_symb
            )

            ts_mean = np.mean(test_losses_symb)
            ts_var = np.var(test_losses_symb)
            ts_std = np.std(test_losses_symb)
            
            print(f"Mean Test loss of {txt}: {ts_mean}")
            print(f"Var Test loss of {txt}: {ts_var}")
            print(f"Std Test loss of {txt}: {ts_std}")
            
            return ts_mean, ts_var, ts_std
        except AssertionError:
            print("Evaluation failed!")
            return np.inf, np.inf, np.inf
        
    
    print("Black-Box fitting \n")
    bb_symb, bb_g_symb, bb_h_symb, exec_time = valid_symb_model(
        config=config,
        model_path_gkan=f"{model_path}/mpnn" if model_type == 'MPNN' else f"{model_path}/llc",
        device=device,
        atol=atol,
        rtol=rtol,
        method=method,
        sample_size = sample_size
    )
    
    print(latex(quantise(bb_symb)))
    ts_mean_bb, ts_var_bb, ts_std_bb = print_symb_error(g_symb=bb_g_symb, h_symb=bb_h_symb)
    
    results_dict["black_box_symb_quant"] = str(quantise(bb_symb))
    results_dict["black_box_symb"] = str(bb_symb)
    results_dict["black_box_symb_test_MAE"] = ts_mean_bb
    results_dict["black_box_symb_test_Var"] = ts_var_bb
    results_dict["black_box_symb_test_Std"] = ts_std_bb
    results_dict["black_box_exec_time"] = exec_time
    
    if eval_model:
        print("Evaluate raw model\n")
        # Loading best model
        if model_type == "MPNN":
            best_model = build_model_from_file_mpnn(
                model_path=model_path,
                message_passing=message_passing,
                include_time=include_time,
                method=method,
                adjoint=adjoint,
                atol=atol,
                rtol=rtol
            )
        elif model_type == "LLC":
            best_model = build_model_from_file_llc(
                model_path=model_path,
                message_passing=message_passing,
                include_time=include_time,
                method=method,
                adjoint=adjoint,
                rtol=rtol,
                atol=atol
            )
        else:
            raise NotImplementedError("Not supported model!")

        tot_params = sum(p.numel() for p in best_model.parameters() if p.requires_grad)
        print(f"Number of model's parameters: {tot_params}\n")
        results_dict["Number of params"] = tot_params

        best_model = best_model.eval()
        ts_mean_model, ts_var_model, ts_std_model = print_symb_error(
            g_symb=best_model.conv.model.g_net,
            h_symb=best_model.conv.model.h_net,
            txt="best model",
            is_symb=False
        )
        
        results_dict["model_test_MAE"] = ts_mean_model
        results_dict["model_test_Var"] = ts_var_model
        results_dict["model_test_Std"] = ts_std_model
    
    with open(f"{model_path}/{res_file_name}", 'w') as file:
        json.dump(results_dict, file, indent=4)   


## LB Losses

### Kuramoto

In [8]:
kur_config = load_config("./configs/config_pred_deriv/config_ic1/config_kuramoto.yml")

KUR = get_test_set(
    dynamics=kur_config['name'],
    device='cuda',
    input_range=kur_config['input_range'],
    **kur_config['integration_kwargs']    
)

g_symb = lambda x: torch.sin(x[:, 1] - x[:, 0]).unsqueeze(-1)
h_symb = lambda x: 2.0 + 0.5 * x[:, 1].unsqueeze(-1)

test_losses = get_symb_test_error(
    g_symb=g_symb,
    h_symb=h_symb,
    test_set=KUR,
    message_passing=True,
    include_time=False,
    is_symb=False
)

ts_mean = np.mean(test_losses)
ts_var = np.var(test_losses)
ts_std = np.std(test_losses)

print(f"Mean Test loss of symbolic formula: {ts_mean}")
print(f"Var Test loss of symbolic formula: {ts_var}")
print(f"Std Test loss of symbolic formula: {ts_std}")



Mean Test loss of symbolic formula: 1.3504788815528931e-05
Var Test loss of symbolic formula: 1.314533673970284e-13
Std Test loss of symbolic formula: 3.625649836884809e-07


### Epidemics

In [9]:
epid_config = load_config("./configs/config_pred_deriv/config_ic1/config_epidemics.yml")

EPID = get_test_set(
    dynamics=epid_config['name'],
    device='cuda',
    input_range=epid_config['input_range'],
    **epid_config['integration_kwargs']    
)

g_symb = lambda x: 0.5*x[:, 1].unsqueeze(-1) * (1 - x[:, 0].unsqueeze(-1))
h_symb = lambda x: x[:, 1].unsqueeze(1) - 0.5 * x[:, 0].unsqueeze(-1)

test_losses = get_symb_test_error(
    g_symb=g_symb,
    h_symb=h_symb,
    test_set=EPID,
    message_passing=True,
    include_time=False,
    is_symb=False
)


ts_mean = np.mean(test_losses)
ts_var = np.var(test_losses)
ts_std = np.std(test_losses)

print(f"Mean Test loss of symbolic formula: {ts_mean}")
print(f"Var Test loss of symbolic formula: {ts_var}")
print(f"Std Test loss of symbolic formula: {ts_std}")


Mean Test loss of symbolic formula: 1.071706852447581e-06
Var Test loss of symbolic formula: 8.008373820613663e-14
Std Test loss of symbolic formula: 2.829907033917133e-07


### Biochemical

In [10]:
bio_config = load_config("./configs/config_pred_deriv/config_ic1/config_biochemical.yml")

BIO = get_test_set(
    dynamics=bio_config['name'],
    device='cuda',
    input_range=bio_config['input_range'],
    **bio_config['integration_kwargs']    
)

g_symb = lambda x: (-0.5*x[:, 1] * x[:, 0]).unsqueeze(-1)
h_symb = lambda x: (1.0 - 0.5 * x[:, 0]).unsqueeze(-1)  + x[:, 1].unsqueeze(-1) 

test_losses = get_symb_test_error(
    g_symb=g_symb,
    h_symb=h_symb,
    test_set=BIO,
    message_passing=True,
    include_time=False,
    is_symb=False
)

ts_mean = np.mean(test_losses)
ts_var = np.var(test_losses)
ts_std = np.std(test_losses)

print(f"Mean Test loss of symbolic formula: {ts_mean}")
print(f"Var Test loss of symbolic formula: {ts_var}")
print(f"Std Test loss of symbolic formula: {ts_std}")


Mean Test loss of symbolic formula: 1.2006473374034006e-06
Var Test loss of symbolic formula: 7.184599599254429e-14
Std Test loss of symbolic formula: 2.6804103415810103e-07


### Population

In [11]:
pop_config = load_config("./configs/config_pred_deriv/config_ic1/config_population.yml")

POP = get_test_set(
    dynamics=pop_config['name'],
    device='cuda',
    input_range=pop_config['input_range'],
    **pop_config['integration_kwargs']    
)

g_symb = lambda x: 0.2*torch.pow(x[:, 1].unsqueeze(-1), 3)
h_symb = lambda x: -0.5 * x[:, 0].unsqueeze(-1) + x[:, 1].unsqueeze(1) 

test_losses = get_symb_test_error(
    g_symb=g_symb,
    h_symb=h_symb,
    test_set=POP,
    message_passing=True,
    include_time=False,
    is_symb=False
)

ts_mean = np.mean(test_losses)
ts_var = np.var(test_losses)
ts_std = np.std(test_losses)

print(f"Mean Test loss of symbolic formula: {ts_mean}")
print(f"Var Test loss of symbolic formula: {ts_var}")
print(f"Std Test loss of symbolic formula: {ts_std}")


Mean Test loss of symbolic formula: 3.735399635237021e-06
Var Test loss of symbolic formula: 4.746857081617248e-13
Std Test loss of symbolic formula: 6.889743886108719e-07


## Symb Reg


### Biochemical

#### IC=1

In [13]:
model_path_mpnn = './saved_models_optuna/model-biochemical-mpnn/biochemical_mpnn_ic1_s5_pd_mult_12/0'
# model_path_mpnn = './saved_models_optuna/model-biochemical-llc/biochemical_llc_2/0'
post_process_mpnn(
    config=bio_config,
    model_path=model_path_mpnn,
    test_set=BIO,
    device='cuda',
    sample_size=10000,
    message_passing=False,
    include_time=False,
    atol=1e-5,
    rtol=1e-5,
    method="dopri5",
    model_type="MPNN",
    res_file_name="post_process_res_final.json"
    
)

Black-Box fitting 

Fitting black-box model with score and 50 iterations
Fitting black-box model with score and 100 iterations
Fitting black-box model with score and 200 iterations
Fitting black-box model with accuracy and 50 iterations
Fitting black-box model with accuracy and 100 iterations
Fitting black-box model with accuracy and 200 iterations
Refitting best model with {'model_selection': 'accuracy', 'param': 200, 'valid_loss': 0.00035483139799907804}
Fitting G_Net...
Execution time: 26.530983 seconds

Fitting H_Net...
Execution time: 30.469593 seconds
\sum_{j}(-0.5*x_i*x_j) - 0.5 x_{i} + 1.0
Mean Test loss of symbolic formula: 0.0003756811493076384
Var Test loss of symbolic formula: 9.394309750620696e-10
Std Test loss of symbolic formula: 3.065013825518687e-05
Evaluate raw model

Number of model's parameters: 2122

Mean Test loss of best model: 0.00038384647147419554
Var Test loss of best model: 2.0909670727479317e-10
Std Test loss of best model: 1.4460176599018187e-05


#### SNR

In [11]:
model_paths = [
    "./saved_models_optuna/model-biochemical-llc/biochemical_llc_70db_3/0",
    "./saved_models_optuna/model-biochemical-llc/biochemical_llc_50db_3/0",
    "./saved_models_optuna/model-biochemical-llc/biochemical_llc_20db_3/0"
]

for model_path in model_paths:
    print(model_path)
    
    post_process_mpnn(
        config=bio_config,
        model_path=model_path,
        test_set=BIO,
        device='cuda',
        sample_size=10000,
        message_passing=False,
        include_time=False,
        atol=1e-5,
        rtol=1e-5,
        method="dopri5",
        model_type="LLC"
    )

./saved_models_optuna/model-biochemical-llc/biochemical_llc_70db_3/0
Black-Box fitting 

Fitting black-box model with score and 50 iterations
Fitting black-box model with score and 100 iterations
Fitting black-box model with score and 200 iterations
Fitting black-box model with accuracy and 50 iterations
Fitting black-box model with accuracy and 100 iterations
Fitting black-box model with accuracy and 200 iterations
Refitting best model with {'model_selection': 'score', 'param': 100, 'valid_loss': 0.0018042810261249542}
Fitting G_Net...
Execution time: 16.123640 seconds

Fitting H_Net...
Execution time: 15.751093 seconds
\sum_{j}(-0.5*x_i*x_j) + \log{\left(2.62 - x_{i} \right)}
Mean Test loss of symbolic formula: 0.0018047988802815478
Var Test loss of symbolic formula: 9.561037944256308e-09
Std Test loss of symbolic formula: 9.778056015515717e-05
Evaluate raw model

Number of model's parameters: 1276

Mean Test loss of best model: 0.0010236709689100583
Var Test loss of best model: 1.53

### Kuramoto

#### IC=1

In [12]:
model_path_mpnn = './saved_models_optuna/model-kuramoto-llc/kuramoto_llc_2/0'

post_process_mpnn(
    config=kur_config,
    model_path=model_path_mpnn,
    test_set=KUR,
    device='cuda',
    sample_size=10000,
    message_passing=False,
    include_time=False,
    atol=1e-5,
    rtol=1e-5,
    method="dopri5",
    model_type="LLC"
)

Black-Box fitting 

Fitting black-box model with score and 50 iterations
Fitting black-box model with score and 100 iterations
Fitting black-box model with score and 200 iterations
Fitting black-box model with accuracy and 50 iterations
Fitting black-box model with accuracy and 100 iterations
Fitting black-box model with accuracy and 200 iterations
Refitting best model with {'model_selection': 'score', 'param': 200, 'valid_loss': 0.0008288701064884663}
Fitting G_Net...
Execution time: 36.346022 seconds

Fitting H_Net...
Execution time: 42.671804 seconds
\sum_{j}(-0.5*sin(x_i - x_j)) + 2.0
Mean Test loss of symbolic formula: 0.0010122246070144076
Var Test loss of symbolic formula: 8.348958345019636e-10
Std Test loss of symbolic formula: 2.8894564099532e-05
Evaluate raw model

Number of model's parameters: 1852

Mean Test loss of best model: 0.004207481785366933
Var Test loss of best model: 2.372292758651333e-08
Std Test loss of best model: 0.00015402249052172


#### SNR

In [12]:
model_paths = [
    "./saved_models_optuna/model-kuramoto-llc/kuramoto_llc_70db_3/0",
    "./saved_models_optuna/model-kuramoto-llc/kuramoto_llc_50db_3/0",
    "./saved_models_optuna/model-kuramoto-llc/kuramoto_llc_20db_3/0"
]

for model_path in model_paths:
    print(model_path)
    
    post_process_mpnn(
        config=kur_config,
        model_path=model_path,
        test_set=KUR,
        device='cuda',
        sample_size=10000,
        message_passing=False,
        include_time=False,
        atol=1e-5,
        rtol=1e-5,
        method="dopri5",
        model_type="LLC"
    )

./saved_models_optuna/model-kuramoto-llc/kuramoto_llc_70db_3/0
Black-Box fitting 

Fitting black-box model with score and 50 iterations
Fitting black-box model with score and 100 iterations
Fitting black-box model with score and 200 iterations
Fitting black-box model with accuracy and 50 iterations
Fitting black-box model with accuracy and 100 iterations
Fitting black-box model with accuracy and 200 iterations
Refitting best model with {'model_selection': 'score', 'param': 200, 'valid_loss': 0.010787715204060078}
Fitting G_Net...
Execution time: 37.103972 seconds

Fitting H_Net...
Execution time: 40.199072 seconds
\sum_{j}(-0.48*sin(x_i - x_j)) + 2.02
Mean Test loss of symbolic formula: 0.014346766906479994
Var Test loss of symbolic formula: 1.651694407352555e-05
Std Test loss of symbolic formula: 0.004064104338415237
Evaluate raw model

Number of model's parameters: 7500

Mean Test loss of best model: 0.03266127035021782
Var Test loss of best model: 5.2339595436386166e-05
Std Test los

### Epidemics

#### IC=1

In [14]:
model_path_mpnn = './saved_models_optuna/model-epidemics-llc/epidemics_llc_2/0'

post_process_mpnn(
    config=epid_config,
    model_path=model_path_mpnn,
    test_set=EPID,
    device='cuda',
    sample_size=10000,
    message_passing=False,
    include_time=False,
    atol=1e-5,
    rtol=1e-5,
    method="dopri5",
    model_type="LLC"
)

Black-Box fitting 

Fitting black-box model with score and 50 iterations
Fitting black-box model with score and 100 iterations
Fitting black-box model with score and 200 iterations
Fitting black-box model with accuracy and 50 iterations
Fitting black-box model with accuracy and 100 iterations
Fitting black-box model with accuracy and 200 iterations
Refitting best model with {'model_selection': 'score', 'param': 200, 'valid_loss': 0.00014084752183407545}
Fitting G_Net...
Execution time: 29.668427 seconds

Fitting H_Net...
Execution time: 23.308801 seconds
\sum_{j}(x_j*(0.5 - 0.5*x_i)) - 0.5 x_{i}
Mean Test loss of symbolic formula: 0.00012937623735827705
Var Test loss of symbolic formula: 1.0037482407200115e-10
Std Test loss of symbolic formula: 1.0018723674800157e-05
Evaluate raw model

Number of model's parameters: 6420

Mean Test loss of best model: 0.0002892174234148115
Var Test loss of best model: 9.847822550342792e-10
Std Test loss of best model: 3.138124049546607e-05


#### SNR

In [13]:
model_paths = [
    "./saved_models_optuna/model-epidemics-llc/epidemics_llc_70db_3/0",
    "./saved_models_optuna/model-epidemics-llc/epidemics_llc_50db_3/0",
    "./saved_models_optuna/model-epidemics-llc/epidemics_llc_20db_3/0"
]

for model_path in model_paths:
    print(model_path)
    post_process_mpnn(
        config=epid_config,
        model_path=model_path,
        test_set=EPID,
        device='cuda',
        sample_size=10000,
        message_passing=False,
        include_time=False,
        atol=1e-5,
        rtol=1e-5,
        method="dopri5",
        model_type="LLC"
    )

./saved_models_optuna/model-epidemics-llc/epidemics_llc_70db_3/0
Black-Box fitting 

Fitting black-box model with score and 50 iterations
Fitting black-box model with score and 100 iterations
Fitting black-box model with score and 200 iterations
Fitting black-box model with accuracy and 50 iterations
Fitting black-box model with accuracy and 100 iterations
Fitting black-box model with accuracy and 200 iterations
Refitting best model with {'model_selection': 'accuracy', 'param': 100, 'valid_loss': 0.0024378716479986906}
Fitting G_Net...
Execution time: 20.159151 seconds

Fitting H_Net...
Execution time: 14.223950 seconds
\sum_{j}(log(-x_j*(log(x_i + 1) - 0.69) + 1)) - 0.49 x_{i}
Mean Test loss of symbolic formula: 0.007499258189151685
Var Test loss of symbolic formula: 7.602420932287265e-07
Std Test loss of symbolic formula: 0.0008719186276417808
Evaluate raw model

Number of model's parameters: 5500

Mean Test loss of best model: 0.007682424193869035
Var Test loss of best model: 5.1271

### Population

#### IC=1

In [15]:
model_path_mpnn = './saved_models_optuna/model-population-llc/population_llc_2/0'

post_process_mpnn(
    config=pop_config,
    model_path=model_path_mpnn,
    test_set=POP,
    device='cuda',
    sample_size=10000,
    message_passing=False,
    include_time=False,
    atol=1e-5,
    rtol=1e-5,
    method="dopri5",
    model_type="LLC"
)

Black-Box fitting 

Fitting black-box model with score and 50 iterations
Fitting black-box model with score and 100 iterations
Fitting black-box model with score and 200 iterations
Fitting black-box model with accuracy and 50 iterations
Fitting black-box model with accuracy and 100 iterations
Fitting black-box model with accuracy and 200 iterations
Refitting best model with {'model_selection': 'score', 'param': 200, 'valid_loss': 0.000729596009477973}
Fitting G_Net...
Execution time: 29.455979 seconds

Fitting H_Net...
Execution time: 32.436466 seconds
\sum_{j}(0.2*x_j**3) - 0.5 x_{i}
Mean Test loss of symbolic formula: 0.0007106232224032283
Var Test loss of symbolic formula: 4.776991871190479e-09
Std Test loss of symbolic formula: 6.911578597679751e-05
Evaluate raw model

Number of model's parameters: 7980

Mean Test loss of best model: 0.0011213955003768206
Var Test loss of best model: 3.652764800198507e-08
Std Test loss of best model: 0.00019112207617641942


#### SNR

In [14]:
model_paths = [
    "./saved_models_optuna/model-population-llc/population_llc_70db_3/0",
    "./saved_models_optuna/model-population-llc/population_llc_50db_3/0",
    "./saved_models_optuna/model-population-llc/population_llc_20db_3/0"
]

for model_path in model_paths:
    print(model_path)
    post_process_mpnn(
        config=pop_config,
        model_path=model_path,
        test_set=POP,
        device='cuda',
        sample_size=10000,
        message_passing=False,
        include_time=False,
        atol=1e-5,
        rtol=1e-5,
        method="dopri5",
        model_type="LLC"
    )

./saved_models_optuna/model-population-llc/population_llc_70db_3/0
Black-Box fitting 

Fitting black-box model with score and 50 iterations
Fitting black-box model with score and 100 iterations
Fitting black-box model with score and 200 iterations
Fitting black-box model with accuracy and 50 iterations
Fitting black-box model with accuracy and 100 iterations
Fitting black-box model with accuracy and 200 iterations
Refitting best model with {'model_selection': 'score', 'param': 50, 'valid_loss': 0.0005758732440881431}
Fitting G_Net...
Execution time: 11.680813 seconds

Fitting H_Net...
Execution time: 12.219921 seconds
\sum_{j}(0.2*x_j**3) - 0.5 x_{i}
Mean Test loss of symbolic formula: 0.0005376727203838527
Var Test loss of symbolic formula: 2.1691011704712328e-10
Std Test loss of symbolic formula: 1.4727868720460652e-05
Evaluate raw model

Number of model's parameters: 1668

Mean Test loss of best model: 0.0013573766918852925
Var Test loss of best model: 1.7540563307112244e-09
Std Tes

### Real Epid

In [7]:
model_path_mpnn = './saved_models_optuna/model-real-epid-llc/real_epid_llc_3/0/llc'

pysr_model = lambda : get_pysr_model(
    model_selection="accuracy",
    n_iterations=200,
    parallelism="serial",
    random_state = 9999,
    deterministic = True
)

mpnn_symb, symb_g, symb_h, _ = fit_mpnn(
    model_path=model_path_mpnn,
    pysr_model=pysr_model,
    sample_size=10000,
    message_passing=False
)

In [8]:
mpnn_symb

\sum_{j}( (x_i - x_j)*exp(-x_j)) + 3.3846776*tanh(x_i + 0.99761075)

In [10]:
from datasets.RealEpidemics import RealEpidemics
from sklearn.metrics import mean_absolute_error

res = {}
model_selections = ["score", "accuracy"]
n_iterations = [50, 100, 200]

real_epid_data = RealEpidemics(
    root = './data_real_epid_covid_int_scaled',
    name = 'RealEpid',
    predict_deriv=False,
    history=1,
    horizon=44,
    scale=True,
    scale_range=(-1, 1)
)

tr_len = real_epid_data[0].y.shape[0]

for mod in model_selections:
    for n_iter in n_iterations:
        print(f"Fitting model with {mod} and {n_iter}")
        pysr_model = lambda : get_pysr_model(
            model_selection=mod,
            n_iterations=n_iter,
            # parallelism="serial",
            # random_state = 9999,
            # deterministic = True
        )
        
        mpnn_symb, symb_g, symb_h, _ = fit_mpnn(
            model_path=model_path_mpnn,
            pysr_model=pysr_model,
            sample_size=10000,
            message_passing=False
        )
        
        g_symb = make_callable(symb_g)
        h_symb = make_callable(symb_h)
        
        symb_model = get_model(
            g = g_symb,
            h = h_symb,
            message_passing=False,
            include_time=False,
            integration_method='rk4'
        )
        
        data_0 = real_epid_data[0]
        y_pred = symb_model(data_0).cpu().detach().numpy()[:int(tr_len*0.8)]
        y_true = real_epid_data[0].y.cpu().detach().numpy()[:int(tr_len*0.8)]
        
        err = mean_absolute_error(y_true.flatten(), y_pred.flatten())
        print(err)
        res[str(mpnn_symb)] = err        

Fitting model with score and 50
0.03440864756703377
Fitting model with score and 100
0.03440649434924126
Fitting model with score and 200
0.03440843150019646
Fitting model with accuracy and 50
0.03442759811878204
Fitting model with accuracy and 100
0.03442646190524101
Fitting model with accuracy and 200
0.03442101925611496


In [11]:
res

{'\\sum_{j}( (x_i - x_j)*exp(-x_j)) + 3.3846576*tanh(x_i + 0.9976123)': 0.03440864756703377,
 '\\sum_{j}( (x_i - x_j)*exp(-x_j)) + 3.38457283272432*tanh(x_i + 0.99761266)': 0.03440649434924126,
 '\\sum_{j}( (x_i - x_j)*exp(-x_j)) + 3.3846574*tanh(x_i + 0.99761236)': 0.03440843150019646,
 '\\sum_{j}( (x_i - x_j)*exp(-x_j)) + 3.384675*tanh(x_i + 0.99760795)': 0.03442759811878204,
 '\\sum_{j}( x_i - x_j) + 3.384769*tanh(x_i + 0.99760836)': 0.03442646190524101,
 '\\sum_{j}( (x_i - x_j)*exp(-x_j)) + 3.3846676*tanh(x_i + 0.99760944)': 0.03442101925611496}